In [ ]:
from pygoridge import RPC, SocketRelay
import numpy as np
import json
from pygoridge.constants import PayloadType
import csv
from numpy import genfromtxt
import pandas as pd
import shap
my_data = genfromtxt('dataset_AI_Random_7.csv',delimiter=',', skip_header=1)

rpc = RPC(SocketRelay("127.0.0.1", 6001))

class memfile:
    def __init__(self):
        self.b = []
    def write(self, b):
        self.b += b
    def asbytes(self):
        return bytes(self.b)


def go_ur_ai_numpy(x):
    mm = memfile()
    np.save(mm, x)
    return np.array(json.loads(rpc("GoUr.InferNumpy", mm.asbytes(), PayloadType.PAYLOAD_RAW)))

with open('dataset_AI_Random_7.csv', 'r') as f:
    csv_reader = csv.reader(f)
    header = []
    for row in csv_reader:
        header = row
        break

#print(my_data)
#print(go_ur_ai_numpy(my_data))

header = [f'{col}_pos_{i+2}' for i, col in enumerate(header)]

my_data_df = pd.DataFrame(my_data)
my_data_df.columns = header

n_samples = 500
bg_values = my_data_df.sample(10)
data_sample = my_data_df.sample(n_samples)

explainer = shap.KernelExplainer(go_ur_ai_numpy, bg_values, link="logit")
shap_values = explainer.shap_values(data_sample, nsamples=n_samples)

In [ ]:
shap.initjs()

shap.force_plot(explainer.expected_value, shap_values[1], data_sample.iloc[1], link="logit")


In [ ]:
df_shap = pd.DataFrame(shap_values)
avg_abs_shap_values = pd.Series(df_shap.abs().mean())
var_shap_values = pd.Series(df_shap.var())
min_shap_values = pd.Series(df_shap.min())
max_shap_values = pd.Series(df_shap.max())

shap_columns = pd.DataFrame({
    'Column': data_sample.columns, 
    'Average Absolute Shap': avg_abs_shap_values, 
    'Variance Shap': var_shap_values, 
    'Min Shap': min_shap_values, 
    'Max Shap': max_shap_values,
}).set_index('Column').sort_values('Average Absolute Shap', ascending=False)
shap_columns

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values, data_sample, link='logit', feature_display_range=slice(None, -50, -1))

In [ ]:
y = go_ur_ai_numpy(data_sample.values)

In [ ]:
y = pd.Series(y)

In [ ]:
good_scores_mask = y > 0.5
good_scores_mask.sum()

In [ ]:
data_sample.reset_index(drop=True)[good_scores_mask]

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values[good_scores_mask], data_sample.reset_index(drop=True)[good_scores_mask], feature_display_range=slice(None, -100, -1))

In [ ]:
shap.decision_plot(explainer.expected_value, shap_values[good_scores_mask], data_sample.reset_index(drop=True)[good_scores_mask], feature_display_range=slice(None, -100, -1), link="logit")